# OBJECTNESS EVALUATION

The purpose of this Notebook is to evaluate the performance of different generic object proposal approaches on high-resultion satellite data

In [1]:
import os
import torch
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader

from models.base.cobnet import COBNet
from helpers.cob.dataset import COBtransform

import datasets
import helpers.utils as utils

## Settings

In [2]:
DATA_PATH = "/home/jovyan/work/DENMARK/256x256"
SAVE_PATH = "/home/jovyan/work/runs/OBJECTNESS"

## Load Data

In [3]:
images_path = os.path.join(DATA_PATH, 'image_sets', 'shapes.txt')
images = [name.replace("\n","") for name in utils.readText(images_path)]

In [25]:
dataset = datasets.getDataset(name = "denmark_shapes", 
                              path = DATA_PATH,
                              images = images,
                              n_classes = 2,
                              transform = None)

sampler = torch.utils.data.RandomSampler(dataset)
loader = DataLoader(dataset, sampler = sampler, batch_size = 1, drop_last = True, num_workers = 1)

Prepare COBNet

In [26]:
means = [0.492, 0.475, 0.430]
stds = [0.176, 0.173, 0.176]
transform = COBtransform(means, stds, 256)

cob_model = COBNet()
cob_model.load_state_dict(torch.load("/home/jovyan/work/runs/COBNET/cp_or.pth.tar"))

<All keys matched successfully>

In [27]:
dataiter = iter(loader)
batch = dataiter.next()

In [88]:
image = batch["images"].numpy()
shapes = batch["shapes"].squeeze().numpy()

In [ ]:
cob_loss_running = 0
cob_mIoU_running = 0

for batch in loader:
    image = batch["images"].numpy()
    shapes = batch["shapes"].squeeze().numpy()
    
    ##### WTP ######
    
    # TODO
    
    ##### COB ######
    image_cob = transform(images = image)[0]
    image_cob = np.stack((image_cob[:,:,0], image_cob[:,:,1], image_cob[:,:,2]), axis = 0)
    cob_tensor = torch.tensor(image_cob[np.newaxis, ...]).float()
    
    cob_model.eval()
    with torch.no_grad():
        out_cob = cob_model(cob_tensor)
    
    cob_preds = out_cob['y_fine'].sigmoid().squeeze()
    
    # eval with BCELoss
    criterion = torch.nn.BCELoss()
    loss = criterion(cob_preds, torch.from_numpy(shapes.astype(np.float32)))
    cob_running += loss.item()
    
    # eval with mIoU
    cob_mask = torch.round(cob_heatmap).numpy() == 1
    shapes_mask = shapes == 1
    union = np.logical_or(cob_mask, shapes_mask).sum()
    intersection = np.logical_and(cob_mask, shapes_mask).sum()
    cob_miou_running += intersection / union
    
    ##### CAM #####
    
    # TODO
    

cob_loss = cob_loss_running / len(loader)
cob_mIoU = cob_mIoU_running / len(loader)

# EXPERIMENTATION ZONE

In [ ]:
mean = [0.492, 0.475, 0.430]
std = [0.176, 0.173, 0.176]

transform = transforms.Compose([
    transforms.ToTensor(), #convert image to Tensor
    transforms.Normalize(mean = mean_new, std = std_new) #normalize Image Tensor
])

In [ ]:
# easy way to determine means and stds for entire dataset - get batch with all images
loader = DataLoader(dataset, sampler = sampler, batch_size = len(dataset), drop_last = True, num_workers = 1)
dataiter = iter(loader)
batch = dataiter.next()
print(np.mean(batch['images'].numpy(), axis = (0, 2, 3)), "\n", np.std(batch['images'].numpy(), axis = (0, 2, 3)))
# sanity check without axis
print(np.mean(batch['images'].numpy()), "\n", np.std(batch['images'].numpy()))